In [1]:
import pandas as pd

## Read and Import Data

In [2]:
cityCodes = pd.read_csv("vigitelData/cityCodes.csv")
cityCodes.head()

,Codigo,Cidade
0,1,aracaju
1,2,belem
2,3,belo horizonte
3,4,boa vista
4,5,campo grande


In [3]:
df = pd.read_csv("vigitelData/all2017.csv")
df = df.set_index('chave')
print(df.shape)
df.head()

(53034, 197)


,replica,ano,mesfim,cidade,regiao,q6,q7,civil,q8a,q8b,...,diab,has,ind_med_has,med_has,trat_med_has,db,ind_med_db,med_db,insulina,trat_med_db
chave,,,,,,,,,,,,,,,,,,,,,
25392337027,4,2017,2,c01,2,49,1,1,8,NaN,...,0,0,0,0,0,0,0,0,0,0
58394937021,13,2017,7,c01,2,73,2,4,5,3.0,...,0,0,0,0,0,0,0,0,0,0
23392497027,4,2017,3,c01,2,40,2,1,6,5.0,...,0,0,0,0,0,0,0,0,0,0
59393257026,11,2017,6,c01,2,80,2,4,1,4.0,...,0,0,0,0,0,0,0,0,0,0
15393537024,1,2017,3,c01,2,45,1,1,4,7.0,...,0,0,0,0,0,0,0,0,0,0


## Choosing Map Variables and Cleaning

#### Map Variables

q74 -> estado de saude (Muito bom[1], Bom[2], Regular[3], Ruim[4], Muito ruim[5], Não sabe [777], Não quis informar[888])

q75 -> pressao alta (Sim[1], Não[2], Não Lembra[777])

q76 -> diabetes (Sim[1], Não[2], Não Lembra[777])



### Diabetes

In [4]:
mapVariables = ['cidade','q76']

In [5]:
import matplotlib.pyplot as plt
df_Diabetes = df[mapVariables]
print(df_Diabetes.shape)
# df_Diabetes

(53034, 2)


In [6]:
totalPerCity = df_Diabetes.groupby('cidade').count()
totalPerCity.columns = ['total']
totalPerCity.head()

,total
cidade,
c01,2025
c02,1920
c03,2008
c04,1674
c05,2032


#### Número de Diabeticos por Cidade

In [7]:
df_Diabetes = df_Diabetes[df_Diabetes['q76']==1]
diabPerCity = df_Diabetes.groupby('cidade').count()
diabPerCity.columns = ['diabeticos']
diabPerCity.head()

,diabeticos
cidade,
c01,223
c02,178
c03,233
c04,137
c05,290


In [8]:
relativeColumn = (diabPerCity['diabeticos'].values/totalPerCity['total'].values)*100
# diabPerCityRel
# totalPerCity['total'].values
diabPerCity['relative'] = relativeColumn
relativePerCity = diabPerCity.drop(['diabeticos'],axis=1)
relativePerCity.head()

,relative
cidade,
c01,11.012346
c02,9.270833
c03,11.603586
c04,8.183990
c05,14.271654


In [9]:
diabPerCity = relativePerCity

#### Decoding City Names

In [10]:
diabPerCity.index = range(1,28)
diabPerCity.head()

,relative
1,11.012346
2,9.270833
3,11.603586
4,8.183990
5,14.271654


In [11]:
cityC = diabPerCity.index
newindex = []
for code in cityC:
    city = cityCodes[cityCodes['Codigo']==code]['Cidade'].values[0]
    newindex.append(city)
diabPerCity.index = newindex
diabPerCity.head()

,relative
aracaju,11.012346
belem,9.270833
belo horizonte,11.603586
boa vista,8.183990
campo grande,14.271654


#### Geocoding as Cidades

In [61]:
import geopandas as gpd
from geopandas.tools import geocode

lats = []
longs = []

cities = diabPerCity.index
for city in cities:
    geo = geocode(city+', Brazil', provider='nominatim')
    longitude = geo['geometry'][0].x
    latitude = geo['geometry'][0].y
    lats.append(latitude)
    longs.append(longitude)
    
diabPerCity['latitude'] = lats
diabPerCity['longitude'] = longs
diabPerCity.head()

,relative,latitude,longitude
aracaju,11.012346,-10.916206,-37.077466
belem,9.270833,-1.450560,-48.468245
belo horizonte,11.603586,-19.922732,-43.945095
boa vista,8.183990,2.820848,-60.671958
campo grande,14.271654,-20.464017,-54.616295


#### Salvando CSV com os Valores

In [62]:
diabPerCity.to_csv('vigitelData/results/diabeticosPorCidade.csv')

### Hipertensão

In [11]:
mapVariables = ['cidade','q75']

In [12]:
import matplotlib.pyplot as plt
df_Hipert = df[mapVariables]
print(df_Hipert.shape)

(53034, 2)


#### Número de Hipertensos por Cidade

In [13]:
df_Hipert = df_Hipert[df_Hipert['q75']==1]
hipertPerCity = df_Hipert.groupby('cidade').count()
hipertPerCity.columns = ['hipertensos']
hipertPerCity.head()

,hipertensos
cidade,
c01,722
c02,586
c03,698
c04,453
c05,847


#### Decoding City Names

In [14]:
hipertPerCity.index = range(1,28)
hipertPerCity.head()

,hipertensos
1,722
2,586
3,698
4,453
5,847


In [15]:
cityC = hipertPerCity.index
newindex = []
for code in cityC:
    city = cityCodes[cityCodes['Codigo']==code]['Cidade'].values[0]
    newindex.append(city)
hipertPerCity.index = newindex
hipertPerCity.head()

,hipertensos
aracaju,722
belem,586
belo horizonte,698
boa vista,453
campo grande,847


#### Geocoding as Cidades

In [16]:
import geopandas as gpd
from geopandas.tools import geocode

lats = []
longs = []

cities = hipertPerCity.index
for city in cities:
    geo = geocode(city+', Brazil', provider='nominatim')
    longitude = geo['geometry'][0].x
    latitude = geo['geometry'][0].y
    lats.append(latitude)
    longs.append(longitude)
    
hipertPerCity['latitude'] = lats
hipertPerCity['longitude'] = longs
hipertPerCity.head()

,hipertensos,latitude,longitude
aracaju,722,-10.916206,-37.077466
belem,586,-1.450560,-48.468245
belo horizonte,698,-19.922732,-43.945095
boa vista,453,2.820848,-60.671958
campo grande,847,-20.464017,-54.616295


#### Salvando CSV com os Valores

In [17]:
hipertPerCity.to_csv('vigitelData/results/hipertensosPorCidade.csv')